In [ ]:
import pandas as pd
import datetime as dt
import plotly.express as px
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../data/processed/emprestimos_exemplares.csv')

df.head()

In [ ]:
len(df.id_emprestimo.drop_duplicates())

In [ ]:
len(df)

In [ ]:
df.dtypes

A diretoria da biblioteca gostaria de entender se a quantidade de empréstimos está diminuindo, aumentando ou permanecendo igual ao decorrer dos últimos anos.

In [ ]:
df['data_emprestimo'].value_counts()

In [ ]:
# convertendo o tipo de data_emprestimo para datetime

df['data_emprestimo'] = pd.to_datetime(df['data_emprestimo'], dayfirst=True, format='ISO8601')

In [ ]:
# verifique qual é a quantidade total de exemplares emprestados por cada ano
# plote um gráfico de linhas.

qtd_por_ano = df['data_emprestimo'].value_counts().reset_index()

qtd_por_ano.rename(
    columns={
        'data_emprestimo': 'data',
        'count': 'quantidade'
    },
    inplace=True
)

qtd_por_ano


In [ ]:
# verifique qual é a quantidade total de exemplares emprestados por cada ano
# plote um gráfico de linhas.

qtd_por_ano_agrupado = qtd_por_ano.groupby(by=qtd_por_ano.data.dt.year).sum(numeric_only=True).reset_index()

qtd_por_ano_agrupado.rename(
    columns={
        'data': 'ano'
    },
    inplace=True
)

# qtd_por_ano_agrupado.columns

fig = px.line(
    qtd_por_ano_agrupado,
    x='ano',
    y='quantidade',
    markers=True,
    title='verifique qual é a quantidade total de exemplares emprestados por cada ano'
)

fig.show()

In [ ]:
qtd_por_ano_agrupado

In [ ]:
sum(qtd_por_ano_agrupado.quantidade)

In [ ]:
# gerenciar a programação de férias dos colaboradores de acordo com os meses de menor demanda;
# programar atividades que não sejam de atendimento ao usuário para períodos específicos de
# menor demanda.

# Portanto, gere uma tabela com a quantidade total de exemplares emprestados por mês e descubra
# quais meses são os que possuem a maior quantidade de empréstimos realizados.

qtd_por_mes_agrupado = qtd_por_ano.groupby(
    by=qtd_por_ano.data.dt.month
    ).sum(numeric_only=True).reset_index()

qtd_por_mes_agrupado.rename(
    columns={
        'data': 'mes'
    },
    inplace=True
)

# qtd_por_mes_agrupado

mes_renomear = {
    1: 'jan',
    2: 'fev',
    3: 'mar',
    4: 'abr',
    5: 'mai',
    6: 'jun',
    7: 'jul',
    8: 'ago',
    9: 'set',
    10: 'out',
    11: 'nov',
    12: 'dez',
}

qtd_por_mes_agrupado['mes'] = qtd_por_mes_agrupado['mes'].map(mes_renomear)

# qtd_por_mes_agrupado.sort_values(by='quantidade',ascending=False)

# Plote um gráfico de linhas.
fig = px.line(
    qtd_por_mes_agrupado,
    x='mes',
    y='quantidade',
    markers=True,
    title='Quantidade total de exemplares emprestados por mês',
)

fig.show()

In [ ]:
qtd_por_mes_agrupado

In [ ]:
qtd_por_ano.data.dt.hour.unique()

In [ ]:
# Além do gerenciamento anual das atividades, a diretoria também necessita que seja planejada 
# uma programação diária das atividades. Por este motivo, verifique quais foram os horários
# com maior quantidade de empréstimos ao longo de um dia inteiro.


qtd_por_hora_agrupado = qtd_por_ano.groupby(
    by=qtd_por_ano.data.dt.hour
    ).sum(numeric_only=True).reset_index()

qtd_por_hora_agrupado.rename(
    columns={
        'data': 'hora'
    },
    inplace=True
)

# qtd_por_hora_agrupado

# Plote um gráfico de barras e analise quais seriam os melhores horários para alocar as demais
# atividades que não sejam de atendimento ao usuário.
fig = px.bar(
    qtd_por_hora_agrupado,
    x='hora',
    y='quantidade',
    title='Quantidade total de exemplares emprestados por hora',
)

fig.show()

In [ ]:
# O objetivo será entender a quantidade de empréstimos a partir das variáveis categóricas do 
# seu conjunto de dados.
# Para explorar os dados, alguns questionamentos serão pertinentes para a diretoria das bibliotecas,
# como:
# Vamos explorar algumas das variáveis categóricas das quais precisaremos extrair mais informações. 
# Elas são:
#   Tipo de vínculo
#   Coleção
#   Biblioteca
#   Classificação geral da CDU

# Ao se analisar uma nova variável, é bem interessante verificar os valores únicos delas.

tipo_vinculo = df['tipo_vinculo_usuario'].unique()

colecao = df['colecao'].unique()

biblioteca = df['biblioteca'].unique()

cdu = df['CDU'].unique()

In [ ]:
tipo_vinculo

In [ ]:
colecao

In [ ]:
biblioteca

In [ ]:
cdu

In [ ]:
df.columns

In [ ]:
# Gere uma tabela de frequência e com o percentual para cada variável.
# Como é um trabalho repetitivo, crie uma função que gere a tabela com os valores.

def gerar_tabela_frequencia(campo):
    df_tabela = pd.DataFrame(df[campo].value_counts().reset_index())

    df_tabela.rename(
        columns={
            'count':'quantidade'
        },
        inplace=True
    )

    df_tabela['porcentagem'] = round((df_tabela['quantidade'] / df_tabela['quantidade'].sum())*100, 10)

    return df_tabela


In [ ]:
# “Como se distribuem os empréstimos de exemplares pelos tipos de vínculo dos usuários?”
# Desta forma, a diretoria poderá entender qual é o público que está utilizando a biblioteca 
# e assim tomar decisões em continuar com a estratégia de negócio atual ou modificá-la.

gerar_tabela_frequencia('tipo_vinculo_usuario')

In [ ]:
# Quais coleções são mais emprestadas?
# Da mesma forma, as coleções. Ranquear as coleções mais emprestadas pelo público, será bastante
# importante para a estratégia atual.

gerar_tabela_frequencia('colecao')

In [ ]:
# Quais são as bibliotecas com mais ou menos quantidade de empréstimos?
# Assim, a diretoria conseguirá entender onde ela deverá melhorar e focar suas iniciativas.

gerar_tabela_frequencia('biblioteca')

In [ ]:
# De quais temas da CDU tem os exemplares mais emprestados?

gerar_tabela_frequencia('CDU')